<a href="https://colab.research.google.com/github/viniciusrpb/natural_language_processing/blob/main/lstm_imdb_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook

import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 10.7MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 1.1MB 46.6MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 194kB 54.7MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 40kB 6.9MB/s 
     |████████████████████████████████| 92kB 13.6MB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 2.6MB 56.2MB/s 
Mounted at /content/gdrive


In [2]:
from fastbook import *
from fastai.text import *
from fastai.text.all import *

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [5]:
(path/'train').ls()

path_train = path/"train"
#path_test = path/"test"

training_set = Path(path_train)

print(training_set)

/root/.fastai/data/imdb/train


### Ajuste do Modelo de Linguagem

In [6]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [7]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj i 've watched this movie twice now on xxup dvd , and both times it did n't fail to impress me with its unique impartial attitude . xxmaj it seems more like a depiction of reality than most other xxmaj hollywood fare , especially on a topic that is still hotly discussed . xxmaj even though it sticks closely with the southern viewpoint , it does n't fail to question it , and in the end the only","xxmaj i 've watched this movie twice now on xxup dvd , and both times it did n't fail to impress me with its unique impartial attitude . xxmaj it seems more like a depiction of reality than most other xxmaj hollywood fare , especially on a topic that is still hotly discussed . xxmaj even though it sticks closely with the southern viewpoint , it does n't fail to question it , and in the end the only sentence"
1,"xxmaj it could have been a lot better . xxmaj less irony and more funny . xxmaj airplane ! , xxmaj naked xxmaj gun , xxmaj top xxmaj secret ! , that 's how this movie should have been . xxmaj an xxmaj abrahams / xxmaj zucker - type movie making fun of bio - pics done by xxmaj apatow and xxmaj kasdan , that would be magic . xxmaj but instead we are treated to a movie that suffers","it could have been a lot better . xxmaj less irony and more funny . xxmaj airplane ! , xxmaj naked xxmaj gun , xxmaj top xxmaj secret ! , that 's how this movie should have been . xxmaj an xxmaj abrahams / xxmaj zucker - type movie making fun of bio - pics done by xxmaj apatow and xxmaj kasdan , that would be magic . xxmaj but instead we are treated to a movie that suffers from"


In [8]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3,metrics=[accuracy, Perplexity()])

In [9]:
#learn.lr_find()

In [10]:
#learn.recorder.plot(skip_end=15)

In [11]:
learn.fit_one_cycle(3, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.232985,4.067201,0.283161,58.393272,54:00
1,4.096335,3.930717,0.296140,50.943501,54:14
2,3.989829,3.863410,0.303243,47.627506,54:16


### É bom salvar os modelos

In [12]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

### Para carregar os modelos e continuar o treinamento

In [13]:
learn = learn.load('1epoch')

#learn.unfreeze()
#learn.fit_one_cycle(5, 2e-3)

### Salva o modelo final

In [14]:
learn.save_encoder('finetuned')

### Adivinha a próxima palavra de uma sentença

In [15]:
sentenca = "I liked this movie because"
n_palavras = 40
n_sentencas = 2

In [17]:
print("\n".join(learn.predict(sentenca, n_palavras, temperature=0.75) for _ in range(n_sentencas)))

i liked this movie because i tried to be a Lieutenant of the West , but i thought the film was a good character . i enjoyed it too . However you 've never been prepared for this . You 've
i liked this movie because that its a great movie . i just think that it was very good , and i would have liked to see how the bad scenes were so stupid . i did n't get that . i can give it


### Classificação de sentenças utilizando LSTM

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=48, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.predict("I really loved that movie, it was awesome!")